<a href="https://colab.research.google.com/github/Saifullah785/langchain-generative-ai-journey/blob/main/Lecture_15_tool_calling_in_langchain/Lecture_15_tool_calling_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# Install necessary libraries for LangChain, HuggingFace integration, and requests.
!pip install -q langchain_huggingface langchain-core requests

In [63]:
# Import necessary classes and functions from LangChain and requests library.
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [64]:
# Import userdata from google.colab to access secrets.
from google.colab import userdata

# Get the Hugging Face API key from Colab secrets, named "hf_token".
hf_api_key = userdata.get("hf_token")

In [65]:
# Initialize the HuggingFaceEndpoint with the Llama-3.1-8B-Instruct model for text generation.
llm = HuggingFaceEndpoint(
    # repo_id="meta-llama/Meta-Llama-3-8B-Instruct", # Example of a commented-out model
    repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1',
)

# Wrap the endpoint in a ChatHuggingFace object for chat-style interaction.
model = ChatHuggingFace(llm=llm)

In [66]:
# Define a tool function using the @tool decorator.
# This function takes two integers, a and b, and returns their product.
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [67]:
# Invoke the multiply tool with inputs a=2 and b=3 and print the result.
print(multiply.invoke({"a": 2, "b": 3}))

6


In [68]:
# Access and display the name of the multiply tool.
multiply.name

'multiply'

In [69]:
# Access and display the description of the multiply tool.
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [70]:
# Access and display the arguments expected by the multiply tool.
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [71]:
# Invoke the chat model with the input 'hi' and display the response.
model.invoke('hi')

AIMessage(content=" Hello! How can I help you today? If you have any questions about a specific topic or just want to chat, feel free to ask. I'm here to provide information and engage in a conversation with you. 😊", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 9, 'total_tokens': 56}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--14f1d46f-4102-4d59-a2bd-ec0ac7737b1a-0', usage_metadata={'input_tokens': 9, 'output_tokens': 47, 'total_tokens': 56})

In [72]:
# Bind the multiply tool to the chat model, creating a new model instance that can use the tool.
llm_with_tools = model.bind_tools([multiply])

In [73]:
# Example of how to invoke the model with tools (currently commented out).
# llm_with_tools.invoke('hi how are you')

AIMessage(content=' Hello! I\'m an artificial intelligence and do not have feelings, but I\'m here to help you. Is there anything you would like to multiply? I can do calculations for you using the "multiply" function.\n\nTo use the "multiply" function, you need to provide me with two numbers to multiply. Here is the format for using the function:\n\n{\n  "function": {\n    "name": "multiply",\n    "parameters": {\n      "a": <number 1>,\n      "b": <number 2>\n    }\n  }\n}\n\nYou can replace <number 1> and <number 2> with the two numbers you would like to multiply. For example, if you want to multiply 3 and 5, the format would look like this:\n\n{\n  "function": {\n    "name": "multiply",\n    "parameters": {\n      "a": 3,\n      "b": 5\n    }\n  }\n}\n\nI hope that helps! Let me know if you have any questions or if you would like to use the "multiply" function.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 258, 'prompt_tokens': 173, 'total_tokens':

In [74]:
# Invoke the model with tools to see if it suggests using the multiply tool for the query "can you multiply 3 with 10".
result = llm_with_tools.invoke('can you multiply 3 with 10')

In [75]:
# Display the tool calls generated by the model's response.
result.tool_calls

[]

In [76]:
# Create a HumanMessage object with the query "can you multiply 3 with 1000".
query = HumanMessage('can you multiply 3 with 1000')

In [77]:
# Create a list containing the initial query message.
messages = [query]

In [78]:
# Display the current list of messages.
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [84]:
# Invoke the model with the messages and display the content of the response.
llm_with_tools.invoke(messages).content

'\n\nSo, 3 multiplied by 1000 is equal to 3000.'

In [79]:
# Invoke the model with the messages and store the response in the 'result' variable.
result = llm_with_tools.invoke(messages)

In [80]:
# Display the model's response.
result

AIMessage(content=' Sure, I can help you with that. To multiply 3 with 1000, I will use the "multiply" function that is available to me. Here is the function call with the appropriate parameters:\n\nmultiply(a: 3, b: 1000)\n\nThis will return the product of 3 and 1000. Let me calculate the result for you.\n\nThe product of 3 and 1000 is 3000.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 181, 'total_tokens': 291}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--634d9248-82c2-4936-8de7-66c9adf5a120-0', usage_metadata={'input_tokens': 181, 'output_tokens': 110, 'total_tokens': 291})

In [81]:
# Print the content of the model's response.
print(result.content)

 Sure, I can help you with that. To multiply 3 with 1000, I will use the "multiply" function that is available to me. Here is the function call with the appropriate parameters:

multiply(a: 3, b: 1000)

This will return the product of 3 and 1000. Let me calculate the result for you.

The product of 3 and 1000 is 3000.


In [82]:
# Append the model's response to the list of messages to continue the conversation history.
messages.append(result)

In [83]:
# Display the updated list of messages.
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' Sure, I can help you with that. To multiply 3 with 1000, I will use the "multiply" function that is available to me. Here is the function call with the appropriate parameters:\n\nmultiply(a: 3, b: 1000)\n\nThis will return the product of 3 and 1000. Let me calculate the result for you.\n\nThe product of 3 and 1000 is 3000.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 181, 'total_tokens': 291}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--634d9248-82c2-4936-8de7-66c9adf5a120-0', usage_metadata={'input_tokens': 181, 'output_tokens': 110, 'total_tokens': 291})]